# Facebook爬蟲
這裡簡單介紹一下爬蟲的邏輯再開始爬進入正題吧！
- 爬取文章內容
 - 貼文人ID
 - 貼文人暱稱
 - 貼文時間
 - 貼文類型
 - 貼文內容
 - 心情：分為Like, Haha, Sad, Wow, Angry...etc.
 - 留言數
 - 分享數
 - 貼文連結


- 爬取留言內容
 - 留言人ID
 - 留言人暱稱
 - 留言時間
 - 回覆對象ID
 - 回覆對象暱稱
 - 貼文連結
 

# 載入使用套件

In [1]:
import pandas as pd
import re, time, requests, datetime, gc
from selenium import webdriver
from bs4 import BeautifulSoup

# 經常用到的函數
Facebook請用戶「登入/註冊」的彈窗會反覆出現，在這裡先寫好嘗試點擊的函數提供使用

In [2]:
# 點擊不要現在註冊帳號
def ClickNotNow():
    try:
        driver.find_element_by_xpath('//a[@id="expanding_cta_close_button"]').click()
    except:
        time.sleep(0.1)

# 截取塗鴉墻貼文資訊並比對先前資料
如果每次都把所有貼文抓下來，無疑會浪費程式效能，在這裡我將載入先前的資料，並將各篇貼文分成以下三類：

- **新貼文**：截取貼文內容與留言
- **舊貼文但有新留言**：截取所有留言但只將新留言加入資料庫
- **舊貼文也無新留言**：不處理

可想而知，如果設定排程每天執行，大多數資料都會是「舊貼文也無新留言」的類型，加入這個判斷機制將大幅節省系統效能

In [3]:
# 截取塗鴉墻上貼文的留言數
def Wall_PostComments(i):
    try:
        Comments = i.find('a', {'data-testid':'UFI2CommentsCount/root'}).text.split(' ',2)[0]
        if 'K' in Comments:
            Comments = int(float(Comments.split('K')[0])*1000)
        else:
            Comments = int(Comments)
    except:
        Comments = 0
    return Comments

# 截取塗鴉墻上貼文的連結
def Wall_PostLink(i):
    Link = 'https://www.facebook.com' + i.find('a',{'class':'_5pcq'}).attrs['href'].split('?',2)[0]
    return Link

# 截取塗鴉墻上貼文的發佈時間
def Wall_PostTime(i):
    try:
        Time = i.find('abbr').attrs['title']
        Time = datetime.datetime.strptime(Time, '%m/%d/%y, %I:%M %p')
        Time = Time.strftime("%Y-%m-%d %H:%M")
    except:
        Time = 'Not Post'
    return Time

def CarwlList(urls, n, Posts):
    CheckList = pd.DataFrame(columns = ['Link','Time','Comments']) 
    for i in urls:
        driver.get(i)        
        for i in range(n):
            try:
                time.sleep(1)
                driver.find_element_by_css_selector('a.pam.uiBoxLightblue.uiMorePagerPrimary').click() # 加載更多貼文的按鈕
            except:
                time.sleep(1)
            # 這裡會跳出要我們登入的大畫面，找到「稍後再說」的按鈕並點擊
            ClickNotNow()
            soup = BeautifulSoup(driver.page_source)
            for i in soup.find_all('div', {'class':'_5pcr userContentWrapper'}):
                CheckList = pd.concat([CheckList, pd.DataFrame(data = [{'Link':Wall_PostLink(i),
                                                                        'Time':Wall_PostTime(i),
                                                                        'Comments':Wall_PostComments(i)}],
                                                               columns = ['Link','Time','Comments'])],
                                       ignore_index = True)
    
    CarwlList = pd.merge(left = CheckList,
                         right = Posts.loc[:,['Link','Comments']],
                         how='left',
                         on='Link',
                         suffixes=('_c', '_p'),
                         indicator=True)
    CarwlList = pd.concat([CarwlList.loc[CarwlList._merge == 'left_only', :], # 新貼文
                           CarwlList.loc[CarwlList.Comments_c > CarwlList.Comments_p, :]], # 舊貼文但有新留言
                          ignore_index=True)
    CarwlList = CarwlList.drop_duplicates(subset = 'Link',
                                          keep= 'first',
                                          inplace = False)
    return CarwlList

# 爬取特定貼文的內容與留言
## 展開貼文與留言

In [4]:
# 關閉彈窗
def ClosePopup():
    time.sleep(1)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(1)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    ClickNotNow()
    driver.find_element_by_xpath('//div[@class="_5pcr userContentWrapper"]//a[@data-testid="UFI2CommentsCount/root"]').click()
    time.sleep(1)
    ClickNotNow()
    time.sleep(0.5)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(0.5)
    ClickNotNow()
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(0.5)
    ClickNotNow()

# 選擇按照New或Oldest來顯示留言(選擇最相關或所有留言都不會會真的顯示所有留言)
def ClickOldest():
    time.sleep(1)
    driver.find_element_by_xpath('//a[@data-testid="UFI2ViewOptionsSelector/link"]').click()
    time.sleep(1)
    try:
        driver.find_element_by_partial_link_text('Comments shown in chronological order with the oldest comments at the top.').click()
    except:
        try:
            driver.find_element_by_partial_link_text('New comments and those with new replies appear at the top.').click()
        except:
            print('Plz, Check this post arragne type!')
    time.sleep(1)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    ClickNotNow()
    time.sleep(1)

# 偵測是否有「更多留言」（第一層），若有則點擊
def MoreComments():
    k = len(driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_0"]'))
    # 卡住的次數超過100次就停止
    l = 0
    while (k != 0) & (l <= 300):
        print('Click More Comments： ' + str(l) + ' times.')
        try:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            # 若彈出註冊視窗點擊not now!
            ClickNotNow()
            driver.find_element_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_0"]').click()
            time.sleep(1)
            k = len(driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_0"]'))
        except:
            time.sleep(0.1)
        finally:
            l += 1
    time.sleep(1)

# 偵測是否有「更多回覆」（第二層），若有則點擊
def MoreReplies():
    k = len(driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_1"]'))
    # 卡住的次數超過100次就停止
    l = 0
    while (k != 0) & (l <= 300):
        l += 1
        print('Click More Replies： ' + str(l) + ' times.')
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        driver.find_element_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_1"]').click()
        time.sleep(0.5)
        k = len(driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_1"]'))
    time.sleep(1)
    
# 偵測是否有「展開留言」（第二層留言內容），若有則點擊
def SeeMore():
    k = len(driver.find_elements_by_xpath('//a[@class="_5v47 fss"]'))
    # 卡住的次數超過100次就停止
    l = 0
    while (k != 0) & (l <= 1000):
        l += 1
        print('Click See More ： ' + str(l) + ' times.')
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        driver.find_element_by_xpath('//a[@class="_5v47 fss"]').click()
        time.sleep(0.5)
        k = len(driver.find_elements_by_xpath('//a[@class="_5v47 fss"]'))
    time.sleep(1)
    
# 組合以上函數，將貼文內容展開
def PostExpand():
    ClosePopup()
    # ClickOldest() # 展開留言會有無法加載更多留言的問題，暫時先不使用
    MoreComments()
    MoreReplies()
    SeeMore()

## 爬取貼文內容與留言資訊

In [5]:
# 貼文留言數
def Post_Name(userContent):
    return userContent.find('img').attrs['aria-label']

# 貼文留言數
def Post_ID(userContent):
    return userContent.find('a', {'class':'_5pb8 o_c3pynyi2g _8o _8s lfloat _ohe'}).attrs['href'].split('/?ref')[0].split('https://www.facebook.com/')[1]

# 貼文時間
def Post_Time(userContent):
    try:
        Time = userContent.find('abbr').attrs['title']
        Time = datetime.datetime.strptime(Time, '%m/%d/%y, %I:%M %p')
        Time = Time.strftime("%Y-%m-%d %H:%M")
    except:
        Time = "Error, Please check this post's condiction!" 
    return Time

# 貼文內容
def Post_Content(userContent):
    try:
        Content = userContent.find('div', {'class':'_5pbx userContent _3576'}).text
    except:
        Content = "There's No Text Content!"
    return Content

# 貼文留言數
def Post_Comments(userContent):
    try:
        CommentCounts = userContent.find('a', {'data-testid':'UFI2CommentsCount/root'}).text.split(' ',2)[0]
        if 'K' in CommentCounts:
            CommentCounts = int(float(CommentCounts.split('K')[0])*1000)
        else:
            CommentCounts = int(CommentCounts)
    except:
        CommentCounts = 0 
    return CommentCounts

# 貼文分享數
def Post_Shares(userContent):
    try:
        ShareCounts = userContent.find('span', {'class':'_355t _4vn2'}).text.split(' ',2)[0]
        if 'K' in ShareCounts:
            ShareCounts = int(float(ShareCounts.split('K')[0])*1000)
        else:
            ShareCounts = int(ShareCounts)
    except:
        ShareCounts = 0
    return ShareCounts

# 貼文按讚心情數
def Post_Likes(userContent):
    try:
        LikeCounts = userContent.find('span', {'data-testid':'UFI2TopReactions/tooltip_LIKE'}).find('a').attrs['aria-label'].split(' ',2)[0]
        if 'K' in LikeCounts:
            LikeCounts = int(float(LikeCounts.split('K')[0])*1000)
        else:
            LikeCounts = int(LikeCounts)
    except:
        LikeCounts = 0
    return LikeCounts

# 貼文愛心心情數
def Post_Loves(userContent):
    try:
        LoveCounts = userContent.find('span', {'data-testid':'UFI2TopReactions/tooltip_LOVE'}).find('a').attrs['aria-label'].split(' ',2)[0]
        if 'K' in LoveCounts:
            LoveCounts = int(float(LoveCounts.split('K')[0])*1000)
        else:
            LoveCounts = int(LoveCounts)
    except:
        LoveCounts = 0
    return LoveCounts

# 貼文哈哈心情數
def Post_Hahas(userContent):
    try:
        HahaCounts = userContent.find('span', {'data-testid':'UFI2TopReactions/tooltip_HAHA'}).find('a').attrs['aria-label'].split(' ',2)[0]
        if 'K' in HahaCounts:
            HahaCounts = int(float(HahaCounts.split('K')[0])*1000)
        else:
            HahaCounts = int(HahaCounts)
    except:
        HahaCounts = 0
    return HahaCounts

# 貼文Wow心情數
def Post_Wows(userContent):
    try:
        WowCounts = userContent.find('span', {'data-testid':'UFI2TopReactions/tooltip_WOW'}).find('a').attrs['aria-label'].split(' ',2)[0]
        if 'K' in WowCounts:
            WowCounts = int(float(WowCounts.split('K')[0])*1000)
        else:
            WowCounts = int(WowCounts)
    except:
        WowCounts = 0
    return WowCounts

# 貼文Sad心情數
def Post_Sads(userContent):
    try:
        SadCounts = userContent.find('span', {'data-testid':'UFI2TopReactions/tooltip_SORRY'}).find('a').attrs['aria-label'].split(' ',2)[0]
        if 'K' in SadCounts:
            SadCounts = int(float(SadCounts.split('K')[0])*1000)
        else:
            SadCounts = int(SadCounts)
    except:
        SadCounts = 0
    return SadCounts

# 貼文Angry心情數
def Post_Angrys(userContent):
    try:
        AngryCounts = userContent.find('span', {'data-testid':'UFI2TopReactions/tooltip_ANGER'}).find('a').attrs['aria-label'].split(' ',2)[0]
        if 'K' in AngryCounts:
            AngryCounts = int(float(AngryCounts.split('K')[0])*1000)
        else:
            AngryCounts = int(AngryCounts)
    except:
        AngryCounts = 0
    return AngryCounts

# 文章內容與互動摘要
def PostInfo(soup):
    # 貼文區
    userContent = soup.find('div', {'class':'_5pcr userContentWrapper'})
    PostContent = pd.DataFrame(data = [{'Name':Post_Name(userContent),
                                        'ID':Post_ID(userContent),
                                        'Time':Post_Time(userContent),
                                        'Content':Post_Content(userContent),
                                        'Comments':Post_Comments(userContent),
                                        'Shares':Post_Shares(userContent),
                                        'Likes':Post_Likes(userContent),
                                        'Loves':Post_Loves(userContent),
                                        'Hahas':Post_Hahas(userContent),
                                        'Wows':Post_Wows(userContent),
                                        'Sads':Post_Sads(userContent),
                                        'Angrys':Post_Angrys(userContent),
                                        'Updatetime':datetime.datetime.now().strftime("%Y-%m-%d %H:%M"),
                                        'Link':driver.current_url}],
                            columns = ['Name', 'ID', 'Time', 'Content', 'Comments', 'Shares', 'Likes', 'Loves', 'Hahas', 'Wows', 'Sads', 'Angrys', 'Updatetime', 'Link'])
    return PostContent

# 留言內容
def Comment_Content(element):
    try:
        Content = element.find('span', {'dir':'ltr'}).text
    except:
        Content = 'img'
    return Content

def CommentsInfo(soup):  
    PostComments = pd.DataFrame()
    userContent = soup.find('div', {'class':'_5pcr userContentWrapper'})
    try:
        for i in userContent.select('ul._7a9a > li'):
            # 先抓留言並放在Comment
            Comment = pd.DataFrame(data=[{'ID':i.find('a', {'class':' _3mf5 _3mg0'}).attrs['data-hovercard'].split('id=',2)[1],
                                          'Name':i.find('img').attrs['alt'],
                                          'Time':datetime.datetime.strptime(i.find('abbr',{'class':'livetimestamp'}).attrs['data-tooltip-content'], '%A, %B %d, %Y at %I:%M %p').strftime("%Y-%m-%d %H:%M"),
                                          'Content':Comment_Content(i),
                                          'RepID':userContent.find('div', {'class':'_5pcp _5lel _2jyu _232_'}).attrs['id'].split(';')[0].split('feed_subtitle_')[-1],
                                          'RepName':userContent.find('img').attrs['aria-label'],
                                          'Link':driver.current_url}],
                                   columns = ['ID', 'Name', 'Time', 'Content', 'RepID', 'RepName', 'Link'])
            PostComments = pd.concat([PostComments, Comment], ignore_index=True)
            # 留言的留言
            for j in i.findAll('div', {'data-testid':'UFI2Comment/root_depth_1'}):
                Comment = pd.DataFrame(data=[{'ID':j.find('a', {'class':' _3mf5 _3mg1'}).attrs['data-hovercard'].split('id=',2)[1],
                                              'Name':j.find('img').attrs['alt'],
                                              'Time':datetime.datetime.strptime(j.find('abbr',{'class':'livetimestamp'}).attrs['data-tooltip-content'], '%A, %B %d, %Y at %I:%M %p').strftime("%Y-%m-%d %H:%M"),
                                              'Content':Comment_Content(j),
                                              'RepID':i.find('a', {'class':' _3mf5 _3mg0'}).attrs['data-hovercard'].split('id=',2)[1],
                                              'RepName':i.find('img').attrs['alt'],
                                              'Link':driver.current_url}],
                                       columns = ['ID', 'Name', 'Time', 'Content', 'RepID', 'RepName', 'Link'])
                PostComments = pd.concat([PostComments, Comment], ignore_index=True)
        PostComments['Updatetime'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
    except:
        print('Crawl Comments Failed!')
    return PostComments

# 更新貼文與留言資訊
將最新抓到的貼文併回Post表格，並以ID與Time作為關鍵(key)值，取最新抓取的資料(Updatetime)

In [6]:
def UpdateData(DateFrame_o,DateFrame_n):
    DataFrame = pd.concat([DateFrame_o, DateFrame_n], ignore_index=True)
    DataFrame = DataFrame.sort_values(by = 'Updatetime', ascending = False)
    DataFrame = DataFrame.drop_duplicates(subset = ['ID', 'Time', 'Content'],
                                          keep= 'first',
                                          inplace = False)
    return DataFrame

# 爬資料
## 讀取先前資料
若在桌面偵測不到檔案，就創造新的dataframe

In [7]:
try:
    Posts = pd.read_pickle('C:/Users/TL_Yu/Desktop/Posts.plk')
except:
    Posts = pd.DataFrame(columns=['Name', 'ID', 'Time', 'Content', 'Comments', 'Shares', 'Likes', 'Loves', 'Hahas', 'Wows', 'Sads', 'Angrys', 'Updatetime', 'Link'])
try:
    Comments = pd.read_pickle('C:/Users/TL_Yu/Desktop/Comments.plk')
except:
    Comments = pd.DataFrame(columns = ['ID', 'Name', 'Time', 'Content', 'RepID', 'RepName', 'Link', 'Updatetime'])

print(Posts.shape)
print(Comments.shape)
urls = ['https://www.facebook.com/tstartel/',
        'https://www.facebook.com/chtmobile/',
        'https://www.facebook.com/taiwanmobile/',
        'https://www.facebook.com/fareastone/',
        'https://www.facebook.com/Aptg.tw/']

(1468, 14)
(219950, 8)


## 產出待爬清單

In [8]:
driver = webdriver.Chrome()
driver.get('https://www.facebook.com/')
time.sleep(1)
driver.find_element_by_partial_link_text('English').click()

In [9]:
CarwlList = CarwlList(urls=urls, n=10, Posts = Posts)
CarwlList

Link              Time  \
0    https://www.facebook.com/tstartel/posts/332434...  2019-06-03 20:30   
1    https://www.facebook.com/tstartel/photos/a.854...  2019-06-03 04:53   
2    https://www.facebook.com/tstartel/photos/a.413...  2019-06-03 02:38   
3    https://www.facebook.com/tstartel/photos/a.854...  2019-06-02 02:54   
36   https://www.facebook.com/tstartel/photos/a.413...  2019-04-21 21:00   
37   https://www.facebook.com/tstartel/photos/a.239...  2019-04-19 20:51   
44   https://www.facebook.com/chtmobile/photos/a.26...  2019-05-30 02:00   
45   https://www.facebook.com/chtmobile/posts/27207...  2019-05-20 02:00   
52   https://www.facebook.com/notes/%E4%B8%AD%E8%8F...  2019-04-18 22:49   
56   https://www.facebook.com/chtmobile/photos/a.26...  2019-04-16 02:00   
57   https://www.facebook.com/notes/%E4%B8%AD%E8%8F...  2019-04-08 20:36   
58   https://www.facebook.com/notes/%E4%B8%AD%E8%8F...  2019-04-02 00:35   
59   https://www.facebook.com/chtmobile/photos/a.26...  2019-03-28 02:00   
67   https://www.facebook.com/notes/%E4%B8%AD%E8%8F...  2019-03-28 01:59   
68   https://www.facebook.com/chtmobile/posts/26214...  2019-03-27 21:00   
69   https://www.facebook.com/chtmobile/photos/a.26...  2019-03-27 02:00   
70   https://www.facebook.com/chtmobile/photos/a.26...  2019-03-14 02:00   
82   https://www.facebook.com/notes/%E4%B8%AD%E8%8F...  2019-02-27 02:12   
83   https://www.facebook.com/notes/%E4%B8%AD%E8%8F...  2019-02-23 04:14   
110  https://www.facebook.com/chtmobile/videos/5589...  2019-02-19 22:17   
111  https://www.facebook.com/notes/%E4%B8%AD%E8%8F...  2019-02-18 00:28   
112   https://www.facebook.com/events/318455062144356/  2019-01-31 03:00   
129  https://www.facebook.com/chtmobile/photos/a.26...  2019-01-23 20:22   
130  https://www.facebook.com/taiwanmobile/photos/a...  2019-06-03 23:30   
131  https://www.facebook.com/taiwanmobile/posts/24...  2019-06-03 04:30   
132  https://www.facebook.com/taiwanmobile/photos/a...  2019-06-02 21:00   
142  https://www.facebook.com/taiwanmobile/posts/24...  2019-05-24 17:00   
143  https://www.facebook.com/taiwanmobile/posts/24...  2019-05-22 17:00   
149  https://www.facebook.com/taiwanmobile/posts/24...  2019-05-21 17:00   
150  https://www.facebook.com/taiwanmobile/posts/24...  2019-05-18 17:00   
..                                                 ...               ...   
279  https://www.facebook.com/Aptg.tw/photos/a.1653...  2019-05-07 20:00   
292  https://www.facebook.com/Aptg.tw/photos/a.1653...  2019-05-07 00:00   
293  https://www.facebook.com/Aptg.tw/photos/a.1653...  2019-05-06 20:00   
294  https://www.facebook.com/Aptg.tw/photos/a.1653...  2019-05-06 00:00   
325  https://www.facebook.com/Aptg.tw/photos/a.1653...  2019-05-02 00:00   
326  https://www.facebook.com/tstartel/videos/23778...  2019-06-01 21:00   
327  https://www.facebook.com/tstartel/photos/a.854...  2019-05-30 21:54   
330  https://www.facebook.com/tstartel/photos/a.854...  2019-05-28 04:24   
331  https://www.facebook.com/tstartel/photos/a.854...  2019-05-27 21:18   
332  https://www.facebook.com/tstartel/videos/23007...  2019-05-26 21:00   
373  https://www.facebook.com/tstartel/photos/a.854...  2019-04-15 02:05   
374  https://www.facebook.com/taiwanmobile/posts/24...  2019-06-01 21:00   
375  https://www.facebook.com/taiwanmobile/posts/24...  2019-05-31 21:00   
378  https://www.facebook.com/taiwanmobile/posts/24...  2019-05-31 21:00   
379  https://www.facebook.com/taiwanmobile/photos/a...  2019-05-30 21:02   
384  https://www.facebook.com/taiwanmobile/posts/24...  2019-05-28 17:00   
420  https://www.facebook.com/fareastone/photos/a.1...  2019-06-02 03:30   
421  https://www.facebook.com/fareastone/posts/2563...  2019-06-01 20:45   
422  https://www.facebook.com/fareastone/photos/a.1...  2019-06-01 03:30   
423  https://www.facebook.com/fareastone/posts/2563...  2019-05-31 03:30   
424  https://www.facebook.com/fareastone/posts/2563...  2019-05-30 20:45   
435  https://www.faceb

如同上面的說明，當CheckLists「merge」欄位為「left_only」時表示該貼文是新貼文；

而當「merge」欄位為「both」，但「Comments_c」的值大於「Comments_p」時，表示該貼文是舊資料但有新留言。

以下將截取這兩類的貼文作為爬取清單

## 執行爬蟲

In [10]:
for i in CarwlList.Link:
    print('Dealing with: ' + i)
    driver.get(i)
    try:
        PostExpand()
        print('Expand Succed!')
        try:
            soup = BeautifulSoup(driver.page_source)
            time.sleep(1)
            nPost = PostInfo(soup)
            nComments = CommentsInfo(soup)
            Posts = UpdateData(DateFrame_o = Posts, DateFrame_n = nPost)
            print('Update PostInfo complete!')
            Comments = UpdateData(DateFrame_o = Comments,DateFrame_n = nComments)
            print('Update CommentsInfo complete!')
        except:
            print('Update Post or CommentsInfo Failed!')
    except:
        print('This post can not expand')
    gc.collect()
    print('Time Log: ' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + '\n\n------------------')

Dealing with: https://www.facebook.com/tstartel/posts/3324347110924646
Click More Comments： 0 times.
Click More Replies： 1 times.
Click More Replies： 2 times.
Expand Succed!
Update PostInfo complete!
Update CommentsInfo complete!
Time Log: 2019-06-04 16:53:44

------------------
Dealing with: https://www.facebook.com/tstartel/photos/a.854369967922385/3322511411108216/
Click More Comments： 0 times.
Click More Replies： 1 times.
Click More Replies： 2 times.
Click More Replies： 3 times.
Click More Replies： 4 times.
Click More Replies： 5 times.
Click More Replies： 6 times.
Click More Replies： 7 times.
Click See More ： 1 times.
Expand Succed!
Update PostInfo complete!
Update CommentsInfo complete!
Time Log: 2019-06-04 16:54:02

------------------
Dealing with: https://www.facebook.com/tstartel/photos/a.413576055335114/3322112707814753/
Click More Comments： 0 times.
Click More Replies： 1 times.
Click More Replies： 2 times.
Click More Replies： 3 times.
Click More Replies： 4 times.
Click More R

C:\Users\TL_Yu\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


Update CommentsInfo complete!
Time Log: 2019-06-04 16:55:18

------------------
Dealing with: https://www.facebook.com/notes/%E4%B8%AD%E8%8F%AF%E9%9B%BB%E4%BF%A1%E8%A1%8C%E5%8B%95%E5%AD%B8%E5%9C%92/%E6%B8%85%E6%98%8E%E9%80%A3%E5%81%87%E8%8F%AF%E7%B7%A8%E6%94%BE%E5%81%87%E7%A7%98%E7%AC%88%E4%B8%AD%E7%8D%8E%E5%90%8D%E5%96%AE/2725299120830475/
This post can not expand
Time Log: 2019-06-04 16:55:22

------------------
Dealing with: https://www.facebook.com/notes/%E4%B8%AD%E8%8F%AF%E9%9B%BB%E4%BF%A1%E8%A1%8C%E5%8B%95%E5%AD%B8%E5%9C%92/hami-%E8%80%81%E5%B8%AB%E7%9A%84%E6%9C%89%E7%8D%8E%E5%BE%B5%E7%AD%94%E4%B8%AD%E7%8D%8E%E5%90%8D%E5%96%AE/2714190525274668/
This post can not expand
Time Log: 2019-06-04 16:55:25

------------------
Dealing with: https://www.facebook.com/chtmobile/photos/a.264830403546609/2625898320773127/
This post can not expand
Time Log: 2019-06-04 16:55:29

------------------
Dealing with: https://www.facebook.com/notes/%E4%B8%AD%E8%8F%AF%E9%9B%BB%E4%BF%A1%E8%A1%8C%E5%8B%95

Expand Succed!
Update PostInfo complete!
Update CommentsInfo complete!
Time Log: 2019-06-04 16:59:15

------------------
Dealing with: https://www.facebook.com/Aptg.tw/photos/a.165348830171434/2481958958510398/
Expand Succed!
Update PostInfo complete!
Update CommentsInfo complete!
Time Log: 2019-06-04 16:59:26

------------------
Dealing with: https://www.facebook.com/Aptg.tw/photos/a.165348830171434/2478035442236083/
This post can not expand
Time Log: 2019-06-04 16:59:30

------------------
Dealing with: https://www.facebook.com/Aptg.tw/photos/a.165348830171434/2453363911369903/
This post can not expand
Time Log: 2019-06-04 16:59:33

------------------
Dealing with: https://www.facebook.com/Aptg.tw/photos/a.165348830171434/2453360004703627/
This post can not expand
Time Log: 2019-06-04 16:59:37

------------------
Dealing with: https://www.facebook.com/Aptg.tw/photos/a.165348830171434/2449437375095890/
This post can not expand
Time Log: 2019-06-04 16:59:40

------------------
Dealing 

Click More Replies： 61 times.
Click More Replies： 62 times.
Click More Replies： 63 times.
Click More Replies： 64 times.
Click More Replies： 65 times.
Click More Replies： 66 times.
Click More Replies： 67 times.
Click More Replies： 68 times.
Click More Replies： 69 times.
Click More Replies： 70 times.
Click More Replies： 71 times.
Click More Replies： 72 times.
Click More Replies： 73 times.
Click More Replies： 74 times.
Click More Replies： 75 times.
Click More Replies： 76 times.
Click More Replies： 77 times.
Click More Replies： 78 times.
Click More Replies： 79 times.
Click More Replies： 80 times.
Click More Replies： 81 times.
Click More Replies： 82 times.
Click More Replies： 83 times.
Click More Replies： 84 times.
Click More Replies： 85 times.
Click More Replies： 86 times.
Click More Replies： 87 times.
Click More Replies： 88 times.
Click More Replies： 89 times.
Click More Replies： 90 times.
Click More Replies： 91 times.
Click More Replies： 92 times.
Click More Replies： 93 times.
Expand Suc

In [14]:
driver.quit()

In [11]:
Posts = Posts.reset_index(drop=True)
Posts = Posts.loc[:,['Name', 'ID', 'Time', 'Content', 'Comments', 'Shares', 'Likes', 'Loves', 'Hahas', 'Wows', 'Sads', 'Angrys', 'Updatetime', 'Link']]
Posts

Name            ID              Time  \
0     亞太電信Gt智慧生活       Aptg.tw  2019-06-02 20:00   
1     亞太電信Gt智慧生活       Aptg.tw  2019-05-31 00:00   
2     亞太電信Gt智慧生活       Aptg.tw  2019-06-01 00:25   
3     亞太電信Gt智慧生活       Aptg.tw  2019-05-27 20:00   
4     亞太電信Gt智慧生活       Aptg.tw  2019-05-28 22:57   
5           遠傳電信    fareastone  2019-05-02 03:30   
6           遠傳電信    fareastone  2019-05-22 20:45   
7           遠傳電信    fareastone  2019-05-27 20:50   
8           遠傳電信    fareastone  2019-05-29 03:30   
9           遠傳電信    fareastone  2019-05-30 20:45   
10          遠傳電信    fareastone  2019-06-01 20:45   
11          遠傳電信    fareastone  2019-06-01 03:30   
12          遠傳電信    fareastone  2019-05-31 03:30   
13    台灣大哥大與你在一起  taiwanmobile  2019-05-31 21:00   
14    台灣大哥大與你在一起  taiwanmobile  2019-05-31 21:00   
15    台灣大哥大與你在一起  taiwanmobile  2019-05-30 21:02   
16          台灣之星      tstartel  2019-04-15 02:05   
17          台灣之星      tstartel  2019-05-26 21:00   
18          台灣之星      tstartel  2019-05-27 21:18   
19          台灣之星      tstartel  2019-05-30 21:54   
20          台灣之星      tstartel  2019-05-28 04:24   
21          台灣之星      tstartel  2019-06-01 21:00   
22    亞太電信Gt智慧生活       Aptg.tw  2019-06-03 00:00   
23    亞太電信Gt智慧生活       Aptg.tw  2019-06-03 20:08   
24          遠傳電信    fareastone  2019-05-23 03:46   
25          遠傳電信    fareastone  2019-06-03 20:45   
26          遠傳電信    fareastone  2019-06-02 21:20   
27    台灣大哥大與你在一起  taiwanmobile  2019-06-03 23:30   
28    台灣大哥大與你在一起  taiwanmobile  2019-06-03 04:30   
29    台灣大哥大與你在一起  taiwanmobile  2019-06-02 21:00   
...          ...           ...               ...   
1450        台灣之星      tstartel  2019-04-29 01:52   
1451        台灣之星      tstartel  2019-04-28 21:00   
1452        台灣之星      tstartel  2019-04-30 02:06   
1453        台灣之星      tstartel  2019-04-30 20:24   
1454        台灣之星      tstartel  2019-05-01 21:25   
1455        台灣之星      tstartel  2019-05-04 20:57   
1456        台灣之星      tstartel  2019-05-04 02:00   
1457        台灣之星      tstartel  2019-05-03 20:37   
1458        台灣之星      tstartel  2019-05-10 02:28   
1459        台灣之星      tstartel  2019-05-07 02:00   
1460        台灣之星      tstartel  2019-05-08 02:00   
1461        台灣之星      tstartel  2019-05-08 21:00   
1462        台灣之星      tstartel  2019-05-10 21:00   
1463        台灣之星      tstartel  2019-05-10 04:56   
1464        台灣之星      tstartel  2019-05-13 01:38   
1465        台灣之星      tstartel  2019-05-11 21:00   
1466        台灣之星      tstartel  2019-05-12 06:20   
1467        台灣之星      tstartel  2019-05-13 22:17   
1468        台灣之星      tstartel  2019-05-14 21:00   
1469        台灣之星      tstartel  2019-05-15 22:11   
1470        台灣之星      tstartel  2019-05-16 21:00   
1471        台灣之星      tstartel  2019-05-17 05:10   
1472        台灣之星      tstartel  2019-05-17 01:00   
1473        台灣之星      tstartel  2019-05-17 21:00   
1474        台灣之星      tstartel  2019-05-18 21:00   
1475        台灣之星      tstartel  2019-05-19 02:00   
1476        台灣之星      tstartel  2019-05-22 02:07   
1477        台灣之星      tstartel  2019-05-23 02:15   
1478        台灣之星      tstartel  2019-05-25 21:01   
1479        台灣之星      tstartel  2019-05-24 21:05   

                                                Content Comments Shares Likes  \
0     股東獨享💝夏普家電特惠齊發第1彈💣 ㊗鴻海45週年㊗生日快樂🎉 亞太電信與夏普攜手聯歡 SH...        5      2    53   
1     小週末的下班，怎麼能直接回家😎 巷弄小店逛不完❗ 一個轉角就是隱藏美食❗ 想不到趴趴造去哪度...        4      0    44   
2                              There's No Text Content!        2      1    44   
3     義氣省錢再出擊‼ 亞太電信就是最挺你💪 超殺方案讓你省到不要不要der~ 🔥299不限速上網...       33      2    75   
4     【院長加持✨亞太電信5G加速器團隊獲肯定💪】 台灣科技新創培育的最高殿堂的「2019 Inn...        3      6    79   
5     有人發覺自己成為遠傳的頭號粉絲嗎？ 這可是要每天來跟小編搏感情🤜🤛 經過臉書核可才能獲得的🎖...      132      8   302   
6     一句話惹怒各職業！ #發洩怒氣拿好禮 　 我是小編，認真的在電信業打拼 雖然滿腔熱血，但是也...      198     32   346   
7     暑假不打手遊說不過去啦！  #文末抽原廠耳機🎁 　 下學期末倒數中~~ 小編先預祝大家期末歐...     1400   1400  1500   
8                              There's No Text Content!   

In [12]:
Comments = Comments.reset_index(drop=True)
Comments = Comments.loc[:,['ID', 'Name', 'Time', 'Content', 'RepID', 'RepName', 'Link', 'Updatetime']]
Comments

ID             Name              Time  \
0       100022733811313              艾喀飛  2019-06-03 12:23   
1       100008036701674              陳國進  2019-06-02 09:02   
2       100000173438302              余雅慧  2019-06-01 17:47   
3       162174107155573       亞太電信Gt智慧生活  2019-06-03 13:37   
4       100000198195362              蔡文豪  2019-05-31 10:31   
5       100000701465888         Jimo Luo  2019-05-31 07:55   
6       100000818895234              蘇文雅  2019-05-30 11:00   
7       162174107155573       亞太電信Gt智慧生活  2019-05-30 12:42   
8       100000818895234              蘇文雅  2019-05-30 17:14   
9       162174107155573       亞太電信Gt智慧生活  2019-05-31 10:44   
10      100000490578917              曹昌漢  2019-05-30 00:39   
11      162174107155573       亞太電信Gt智慧生活  2019-05-30 12:42   
12      100002470664192       Chun Ju Wu  2019-05-28 21:15   
13      100008036701674              陳國進  2019-05-28 13:01   
14      100000065800888              賴小翰  2019-05-28 13:59   
15      100008036701674              陳國進  2019-05-28 15:08   
16      100000065800888              賴小翰  2019-05-29 23:51   
17      100000065800888              賴小翰  2019-05-29 23:54   
18      100008036701674              陳國進  2019-05-30 06:17   
19      100008036701674              陳國進  2019-05-30 06:17   
20      100008036701674              陳國進  2019-05-30 06:18   
21      100011684129627  Sheng Jen Huang  2019-05-28 14:13   
22      162174107155573       亞太電信Gt智慧生活  2019-05-29 14:15   
23      100004803518108              金明倆  2019-05-28 13:17   
24      162174107155573       亞太電信Gt智慧生活  2019-05-29 14:15   
25      100024104980789              孫天龍  2019-06-03 19:53   
26      100004746909420        Bryan Lin  2019-05-28 11:08   
27      162174107155573       亞太電信Gt智慧生活  2019-06-03 13:36   
28      100008036701674              陳國進  2019-06-02 16:47   
29      100022733811313              艾喀飛  2019-06-03 19:45   
...                 ...              ...               ...   
221178  100008036701674              陳國進  2019-05-27 17:41   
221179  360044337354953             台灣之星  2019-05-27 13:16   
221180  100008036701674              陳國進  2019-05-27 16:55   
221181  100008036701674              陳國進  2019-05-26 12:44   
221182  100008036701674              陳國進  2019-05-25 14:24   
221183  360044337354953             台灣之星  2019-05-25 20:10   
221184  100008036701674              陳國進  2019-05-27 17:36   
221185       1635861603              連佩婷  2019-05-25 14:42   
221186  100008036701674              陳國進  2019-05-27 18:33   
221187  100008036701674              陳國進  2019-05-25 19:52   
221188  100008036701674              陳國進  2019-05-26 17:26   
221189  100000293657248              賴清溪  2019-05-26 13:16   
221190  100008036701674              陳國進  2019-05-27 17:36   
221191  360044337354953             台灣之星  2019-05-26 20:25   
221192  100008036701674              陳國進  2019-05-27 21:46   
221193  100008036701674              陳國進  2019-05-27 19:49   
221194  360044337354953             台灣之星  2019-05-27 20:10   
221195  360044337354953             台灣之星  2019-05-27 08:38   
221196  360044337354953             台灣之星  2019-05-26 13:28   
221197  100008036701674              陳國進  2019-05-27 06:10   
221198  360044337354953             台灣之星  2019-05-26 21:43   
221199  360044337354953             台灣之星  2019-05-27 18:30   
221200  360044337354953             台灣之星  2019-05-27 17:52   
221201  360044337354953             台灣之星  2019-05-25 13:33   
221202  100000133092219              莊永新  2019-05-25 17:40   
221203  100008036701674              陳國進  2019-05-25 12:42   
221204  360044337354953             台灣之星  2019-05-26 17:35   
221205  360044337354953             台灣之星  2019-05-26 18:30   
221206  100008036701674              陳國進  2019-05-26 21:32   
221207  360044337354953             台灣之星  2019-05-27 21:23   

                                                  Content  \
0                                      請問網站標的價錢是打折錢還是打折後？   
1                                                 這個封面

# 保存資料

In [13]:
Posts.to_pickle('C:/Users/TL_Yu/Desktop/Posts.plk')
Comments.to_pickle('C:/Users/TL_Yu/Desktop/Comments.plk')
Posts.to_csv('C:/Users/TL_Yu/Desktop/Posts.csv')
Comments.to_csv('C:/Users/TL_Yu/Desktop/Comments.csv')

# To Do List
- 故障(無法expand的貼文要避免加入貼文清單或者加入後，要讓下次再來爬文，不能直接跳過。)
 - 評估將加載更多留言放在同一個function中

# 維護測試區

In [60]:
#driver = webdriver.Chrome()
#driver.get('https://www.facebook.com/')
#time.sleep(1)
#driver.find_element_by_partial_link_text('English').click()

In [61]:
#driver.get('https://www.facebook.com/tstartel/videos/1100239263492734/?permPage=1')
#time.sleep(0.7)
#PostExpand()

# 其他

In [18]:
# 原本想要登入的方式獲取連結，但FB會偵測異常行為，暫時先不登入
# 關閉Chrome的「通知」提醒
#chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument("--disable-notifications")

#driver = webdriver.Chrome(options=chrome_options)
#url = 'https://www.facebook.com/'
#driver.get(url)
#time.sleep(3)

# 啟動瀏覽器並登入Facebook
#username = driver.find_element_by_id('email')
#username.send_keys('61034b001@gms.ndhu.edu.tw')
#passwd=driver.find_element_by_id('pass')
#passwd.send_keys('19920309')
#button=driver.find_element_by_id('loginbutton')
#button.click()